<a href="https://colab.research.google.com/github/mjain2/csci544-group32/blob/main/ridge_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##  Bayesian Linear Ridge Regression & AES

The goal with this notebook is to establish a baseline with classic machine learning methodology, particularly with bayesian linear ridge regression

# Setting up train and test data
The first part is consistent across all baselines to make sure that we are using the same features (word2vec) for input.
https://www.kaggle.com/molleejain/group-32-feature-extraction-preprocessing-base/edit

In [ ]:
import numpy as np 
import pandas as pd 
import os, gc, re, warnings
warnings.filterwarnings("ignore")

# import gensim
# from gensim.models import Word2Vec, KeyedVectors

In [ ]:
train = pd.read_csv("/kaggle/input/feedback-prize-english-language-learning/train.csv")
test = pd.read_csv("/kaggle/input/feedback-prize-english-language-learning/test.csv")
# df = pd.concat([train,test],ignore_index=True)

train.head()

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5


In [ ]:
train['full_text']=train['full_text'].str.lower()
test['full_text']=test['full_text'].str.lower()
train['full_text']=train['full_text'].apply(lambda x: re.sub(r'[^ A-Za-z]+',' ', x))
train['full_text']=train['full_text'].apply(lambda x: re.sub(r' +',' ', x))

test['full_text']=test['full_text'].apply(lambda x: re.sub(r'[^ A-Za-z]+',' ', x))
test['full_text']=test['full_text'].apply(lambda x: re.sub(r' +',' ', x))

In [ ]:
target_cols = ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions',]

In [ ]:
# split into train/validation (80-20)
from sklearn.model_selection import train_test_split

train, validation, train_labels, validation_labels = train_test_split(train, train[target_cols], test_size=0.20)

In [ ]:
# train, train_labels
train = train.reset_index()
validation = validation.reset_index()

In [ ]:
print(train.shape)
print(validation.shape)
print(test.shape)

(3128, 9)
(783, 9)
(3, 2)


In [ ]:
train.head()

,index,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,1518,74E468FCE398,it has been said that first impressions are al...,3.5,2.5,3.5,3.5,2.5,3.0
1,2109,9E49399B42B4,some people belive that students should be req...,3.5,3.5,3.0,2.5,2.5,3.0
2,471,23958A3106EA,the work in group will always have an excellen...,4.0,4.0,4.0,4.0,3.5,4.5
3,2335,AD72A536B54A,i d kinda have to agree with micky here least ...,2.5,3.0,2.5,2.0,3.0,2.5
4,627,2F7B0CE01B4B,people who take decisions with guidance from e...,3.5,3.0,4.0,4.0,4.0,4.0


,cohesion,syntax,vocabulary,phraseology,grammar,conventions
1143,3.5,3.5,3.5,4.0,4.0,3.5
1706,3.0,2.5,2.5,2.0,2.5,3.0
2813,3.0,2.5,3.5,3.5,2.5,4.0
484,3.5,3.5,3.5,3.5,4.0,3.0
2634,3.5,3.5,3.5,3.5,2.5,3.5
...,...,...,...,...,...,...
2205,3.5,3.0,3.0,3.0,3.0,3.0
2503,2.5,3.0,3.0,2.5,2.5,3.5
271,3.0,2.5,4.0,3.5,3.0,3.0
523,3.0,3.0,2.5,2.5,3.0,3.0


In [ ]:
import sys
sys.path.append('../input/iterativestratification')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
FOLDS = 5 # CHANGED
skf = MultilabelStratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=42)
for i,(train_index, val_index) in enumerate(skf.split(train,train[target_cols])):
#     print(train.loc[val_index, 'FOLD'])
#     print(train.loc[val_index,'index'])
    train.loc[val_index,'FOLD'] = i
print('Train samples per fold:')
train.FOLD.value_counts()

Train samples per fold:


4.0    626
1.0    626
2.0    626
3.0    625
0.0    625
Name: FOLD, dtype: int64

In [ ]:
train.head()

,index,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions,FOLD
0,1518,74E468FCE398,it has been said that first impressions are al...,3.5,2.5,3.5,3.5,2.5,3.0,3.0
1,2109,9E49399B42B4,some people belive that students should be req...,3.5,3.5,3.0,2.5,2.5,3.0,4.0
2,471,23958A3106EA,the work in group will always have an excellen...,4.0,4.0,4.0,4.0,3.5,4.5,1.0
3,2335,AD72A536B54A,i d kinda have to agree with micky here least ...,2.5,3.0,2.5,2.0,3.0,2.5,3.0
4,627,2F7B0CE01B4B,people who take decisions with guidance from e...,3.5,3.0,4.0,4.0,4.0,4.0,1.0


In [ ]:
# drop index columns
train = train.drop(columns=["index"])

In [ ]:
# train_data = [sentence.split() for sentence in train['full_text']]
# model = Word2Vec(sentences=train_data, vector_size=100, window=5, min_count=1)
# # model.save('word2vec.model')

In [ ]:
# X_train = [[model.wv[token] for token in sentence.split()] for sentence in train['full_text']]
# X_test = [[model.wv[token] for token in sentence.split() if token in model.wv] for sentence in test['full_text']]

# Y_train = train[['cohesion','syntax','vocabulary','phraseology','grammar','conventions']]

# print(f'X train shape: {len(X_train)}, {len(X_train[0])}, {len(X_train[0][0])}')
# print(f'y train shape: {Y_train.shape}')
# print(f'X test shape: {len(X_test)}, {len(X_test[0])}, {len(X_test[0][0])}')

# Generate Embeddings

In [ ]:
from transformers import AutoModel,AutoTokenizer
import torch
import torch.nn.functional as F
from tqdm import tqdm

In [ ]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state.detach().cpu()
    input_mask_expanded = (
        attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    )
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(
        input_mask_expanded.sum(1), min=1e-9
    )

In [ ]:
BATCH_SIZE = 4

class EmbedDataset(torch.utils.data.Dataset):
    def __init__(self,df):
        self.df = df.reset_index(drop=True)
    def __len__(self):
        return len(self.df)
    def __getitem__(self,idx):
        text = self.df.loc[idx,"full_text"]
        tokens = tokenizer(
                text,
                None,
                add_special_tokens=True,
                padding='max_length',
                truncation=True,
                max_length=MAX_LEN,return_tensors="pt")
        tokens = {k:v.squeeze(0) for k,v in tokens.items()}
        return tokens

ds_tr = EmbedDataset(train)
embed_dataloader_tr = torch.utils.data.DataLoader(ds_tr,\
                        batch_size=BATCH_SIZE,\
                        shuffle=False)
ds_va = EmbedDataset(validation)
embed_dataloader_te = torch.utils.data.DataLoader(ds_va,\
                        batch_size=BATCH_SIZE,\
                        shuffle=False)

ds_te = EmbedDataset(test)
embed_dataloader_te_old = torch.utils.data.DataLoader(ds_te,\
                        batch_size=BATCH_SIZE,\
                        shuffle=False)

In [ ]:
ds_tr.df.head()

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions,FOLD
0,74E468FCE398,it has been said that first impressions are al...,3.5,2.5,3.5,3.5,2.5,3.0,3.0
1,9E49399B42B4,some people belive that students should be req...,3.5,3.5,3.0,2.5,2.5,3.0,4.0
2,23958A3106EA,the work in group will always have an excellen...,4.0,4.0,4.0,4.0,3.5,4.5,1.0
3,AD72A536B54A,i d kinda have to agree with micky here least ...,2.5,3.0,2.5,2.0,3.0,2.5,3.0
4,2F7B0CE01B4B,people who take decisions with guidance from e...,3.5,3.0,4.0,4.0,4.0,4.0,1.0


In [ ]:
ds_va.df.head()

,index,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,493,25597135F582,student s that don t want to be in high school...,3.5,3.5,4.0,3.5,3.5,3.5
1,3452,EC7A73638669,some school districts offer programs that allo...,4.0,4.0,4.0,4.0,3.0,3.5
2,1690,80AD3F379BD1,i think that a act of kindnees is great for pe...,2.5,2.0,3.0,3.0,2.5,2.0
3,1542,7680EF5AD3CB,i believe students should not be required to t...,3.5,3.5,3.0,4.0,4.0,3.5
4,1980,960D7C072801,hello mr manager i would like to vist your wor...,3.0,3.5,3.5,3.0,4.0,3.0


In [ ]:
tokenizer = None
MAX_LEN = 640

def get_embeddings(MODEL_NM='', MAX=640, BATCH_SIZE=4, verbose=True):
    global tokenizer, MAX_LEN
    DEVICE="cuda" # can change to cuda if torch.cuda enabled, make sure GPU enabled under acceleration
    model = AutoModel.from_pretrained( MODEL_NM )
    tokenizer = AutoTokenizer.from_pretrained( MODEL_NM )
    MAX_LEN = MAX
    
    model = model.to(DEVICE)
    model.eval()
    all_train_text_feats = []
    for batch in tqdm(embed_dataloader_tr,total=len(embed_dataloader_tr)):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        with torch.no_grad():
            model_output = model(input_ids=input_ids,attention_mask=attention_mask)
        sentence_embeddings = mean_pooling(model_output, attention_mask.detach().cpu())
        # Normalize the embeddings
        sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
        sentence_embeddings =  sentence_embeddings.squeeze(0).detach().cpu().numpy()
        all_train_text_feats.extend(sentence_embeddings)
    all_train_text_feats = np.array(all_train_text_feats)
    if verbose:
        print('Train embeddings shape',all_train_text_feats.shape)
        
    te_text_feats = []
    for batch in tqdm(embed_dataloader_te,total=len(embed_dataloader_te)):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        with torch.no_grad():
            model_output = model(input_ids=input_ids,attention_mask=attention_mask)
        sentence_embeddings = mean_pooling(model_output, attention_mask.detach().cpu())
        # Normalize the embeddings
        sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
        sentence_embeddings =  sentence_embeddings.squeeze(0).detach().cpu().numpy()
        te_text_feats.extend(sentence_embeddings)
    te_text_feats = np.array(te_text_feats)
    if verbose:
        print('Test embeddings shape',te_text_feats.shape)
        
    return all_train_text_feats, te_text_feats

In [ ]:
MODEL_NM = '../input/huggingface-deberta-variants/deberta-large/deberta-large'
all_train_text_feats, te_text_feats = get_embeddings(MODEL_NM)

Some weights of the model checkpoint at ../input/huggingface-deberta-variants/deberta-large/deberta-large were not used when initializing DebertaModel: ['lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.bias', 'config', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.bias']
- This IS expected if you are initializing DebertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 782/782 [06:27<00:00,  2.02it/s]


Train embeddings shape (3128, 1024)


100%|██████████| 196/196 [01:37<00:00,  2.02it/s]

Test embeddings shape (783, 1024)


In [ ]:
all_train_text_feats.shape

(3128, 1024)

In [ ]:
# write dataset to pickle file for easy use across models:

import pickle

dataset_dict = {"X_train": all_train_text_feats, "X_test": te_text_feats}

with open('dataset_dict.pickle', 'wb') as file:
    pickle.dump(dataset_dict, file)

# Bayesian Linear Ridge Regression (BLRR) setup

In [ ]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import mean_squared_error as MSE

# clf = linear_model.BayesianRidge()

In [ ]:
preds = []
scores = []
def comp_score(y_true,y_pred):
    rmse_scores = []
    for i in range(len(target_cols)):
        rmse_scores.append(np.sqrt(mean_squared_error(y_true[:,i],y_pred[:,i])))
    return np.mean(rmse_scores)

#for fold in tqdm(range(FOLDS),total=FOLDS):
for fold in range(FOLDS):
    print('#'*5) # CHANGED
    print('### Fold',fold+1)
    print('#'*5) # CHANGED
    
    train_ = train[train["FOLD"]!=fold]
    dev_ = train[train["FOLD"]==fold]
    
    tr_text_feats = all_train_text_feats[list(train_.index),:]
    ev_text_feats = all_train_text_feats[list(dev_.index),:]
    
    ev_preds = np.zeros((len(ev_text_feats),6))
    test_preds = np.zeros((len(te_text_feats),6))
    for i,t in enumerate(target_cols):
        print(t,', ',end='')
#         clf = SVR(C=1)
        clf = linear_model.BayesianRidge()
        clf.fit(tr_text_feats, train_[t].values)
        ev_preds[:,i] = clf.predict(ev_text_feats)
        test_preds[:,i] = clf.predict(te_text_feats)
    print()
    score = comp_score(dev_[target_cols].values,ev_preds)
    scores.append(score)
    print("Fold : {} RSME score: {}".format(fold,score))
    preds.append(test_preds)
    
print('#'*5)
print('Overall CV RSME =',np.mean(scores))

#####
### Fold 1
#####
cohesion , syntax , vocabulary , phraseology , grammar , conventions , 
Fold : 0 RSME score: 0.47941131989652724
#####
### Fold 2
#####
cohesion , syntax , vocabulary , phraseology , grammar , conventions , 
Fold : 1 RSME score: 0.47670377799562225
#####
### Fold 3
#####
cohesion , syntax , vocabulary , phraseology , grammar , conventions , 
Fold : 2 RSME score: 0.4657889715993801
#####
### Fold 4
#####
cohesion , syntax , vocabulary , phraseology , grammar , conventions , 
Fold : 3 RSME score: 0.48079192568095225
#####
### Fold 5
#####
cohesion , syntax , vocabulary , phraseology , grammar , conventions , 
Fold : 4 RSME score: 0.46100910858703076
#####
Overall CV RSME = 0.4727410207519026


Overall CV RSME = 0.4823354715043138

Last ran: 11/10

In [ ]:
# predict on test data
preds = np.zeros((len(te_text_feats),6))
for i,t in enumerate(target_cols):
    clf = linear_model.BayesianRidge()
    clf.fit(all_train_text_feats, train_labels[t])
    print("Predicting on validation set for " + str(t))
    preds[:, i] = clf.predict(te_text_feats)

Predicting on validation set for cohesion
Predicting on validation set for syntax
Predicting on validation set for vocabulary
Predicting on validation set for phraseology
Predicting on validation set for grammar
Predicting on validation set for conventions


In [ ]:
score = comp_score(validation_labels.values, preds)
print(score)

0.4733969167634382


## Second test - ignore below for now, all commented out

In [ ]:
# class ELLDataset(Dataset):
#     def __init__(self, X, Y=False, transform=False, max_len=420, test=False):
#         self.X = X
#         self.Y = Y
#         self.transform = transform
#         self.max_len = max_len
#         self.test = test
        
#     def __len__(self):
#         return len(self.X)

#     def __getitem__(self, idx):
#         features = torch.tensor(np.array(self.X[idx]))
#         if self.transform == "avg":
#             features = torch.mean(features, 1, True)  # collapsing word2vec vector per token

#         if len(features) < self.max_len:
#             features = F.pad(features, (0, 0, 0, self.max_len-features.shape[0]))    
#         if len(features) > self.max_len:
#             features = features[:self.max_len]               
        
#         features = features.permute(1, 0)
        
#         if self.test == True:
#             return features.float()
#         else:
#             label = self.Y[idx]
#             label = torch.tensor(label)
#             return features.float(), label.float()

In [ ]:
# # Hyperparameters
# batch_size = 64
# max_len = 420
# emb_dim = 100
# learning_rate = 0.01
# epochs = 20
# dropout_p = 0.3

In [ ]:
# Y_train = Y_train.values
# train_dataset = ELLDataset(X_train, Y_train, max_len=max_len)
# # dev_dataset = ELLDataset(X_dev, y_dev, max_len=max_len)
# test_dataset = ELLDataset(X_test, max_len=max_len, test=True)


In [ ]:
# print(len(train_dataset))
# print(train_dataset[0][0].shape)  # feature's shape
# print(train_dataset[0][1])  # label
# print(train_dataset[0][0].dtype)
# print(test_dataset[0].shape)
# train_dataset[0]

In [ ]:
# train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
# print(len(train_dataloader))


In [ ]:
# class CNNExtractor(nn.Module):
#     def __init__(self, emb_dim):
#         super().__init__()
#         self.conv1 = nn.Conv1d(emb_dim, emb_dim//2, 5, 1)
#         self.conv2 = nn.Conv1d(emb_dim//2, emb_dim//4, 5, 1)
#         self.pool = nn.MaxPool1d(2)
#         self.dropout = nn.Dropout(dropout_p)
#         self.fc1 = nn.Linear(emb_dim//4 * 102, 120)
#         self.fc2 = nn.Linear(120, 6)

#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.dropout(x)
#         x = self.pool(F.relu(self.conv2(x)))
#         x = self.dropout(x)
#         x = torch.flatten(x, 1) # flatten all dimensions except batch
#         x = F.relu(self.fc1(x))
#         x = self.fc2(x)
#         return x
    
#     def extract(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.pool(F.relu(self.conv2(x)))
#         x = torch.flatten(x, 1) # flatten all dimensions except batch
#         x = F.relu(self.fc1(x))
#         return x # [batch, 120]

In [ ]:
# def extractFeatures(dataset):
# #     model = CNNExtractor(emb_dim)
# #     model.load_state_dict(torch.load(model_name))
# #     model.eval()
    
#     train_features = []
#     with torch.no_grad():
#         for x in dataset:
# #             train_features.append(model.extract(x.view(1, emb_dim, max_len)).numpy())
#             train_features.append(x[0].reshape(emb_dim, max_len).numpy())
#     return np.array(train_features)

In [ ]:
# train_cnn_features = extractFeatures(train_dataset)


In [ ]:
# train2 = train_cnn_features.reshape(3911, 100*420)

In [ ]:
# test_cnn_features = extractFeatures(test_dataset)

In [ ]:
# train2.shape

In [ ]:
# first = Y_train

In [ ]:
# clf.fit(train2, Y_train)